In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
plt.rcParams["figure.figsize"] = (15, 5)
from pmdarima import auto_arima, model_selection, ARIMA
from pmdarima.arima.utils import ndiffs
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from datetime import datetime, timedelta, date
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
import joblib

In [64]:
country="Spain"
filename = './models/' + country + 'SARIMAXmodel.pkl'
filename= "/home/dsc/proyecto/SpainSARIMAXmodel.pkl"
datecol = 'date'

In [4]:
model=joblib.load(filename)

<bound method SARIMAXResults.summary of <statsmodels.tsa.statespace.sarimax.SARIMAXResults object at 0x7fdc250e76a0>>

In [53]:
model.fittedvalues.index[-1]

Timestamp('2020-08-22 00:00:00')

In [66]:
lastdateexog=pd.read_csv("/home/dsc/proyecto/exogenas/exogenas.csv", parse_dates=[datecol], index_col=[datecol])

In [69]:
lastdateexog.index[-1]

Timestamp('2020-09-06 00:00:00')

In [70]:
lastdateexog.index[-1] - model.fittedvalues.index[-1]

Timedelta('15 days 00:00:00')

In [72]:
lastdateendog=pd.read_csv("/home/dsc/proyecto/exogenas/endogenous.csv", parse_dates=[datecol], index_col=[datecol])

In [75]:
lastdateendog.index[-1]

Timestamp('2020-09-06 00:00:00')

In [76]:
lastdateendog.index[-1] - model.fittedvalues.index[-1]

Timedelta('15 days 00:00:00')